In [1]:
# Set Up 
import pandas as pd
import numpy as np
import wget
import sh
import simplejson as json

In [2]:
# Download the data
wget.download("http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Musical_Instruments.json.gz")
wget.download("http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Musical_Instruments.json.gz")

'meta_Musical_Instruments.json.gz'

In [3]:
# Unzip 
sh.gunzip("Musical_Instruments.json.gz")
sh.gunzip("meta_Musical_Instruments.json.gz")

In [4]:
# Normalize the data to create a .csv file
F1 = "Musical_Instruments.json"
DF1 = pd.io.json.json_normalize([json.loads(line) for line in open(F1)])

F2 = "meta_Musical_Instruments.json"
DF2 = pd.io.json.json_normalize([json.loads(line) for line in open(F2)])

In [5]:
# Create the .csv file 
DF1.to_csv("Musical_Instruments.csv", index=False)
DF2.to_csv("meta_Musical_Instruments.csv", index=False)

In [56]:
# Reading the .csv file with pandas 
DF1 = pd.read_csv("Musical_Instruments.csv")
DF2 = pd.read_csv("meta_Musical_Instruments.csv")

In [57]:
# Merging the review data and meta data on 'asin'
DF3 = pd.merge(DF1, DF2, how='inner', left_on='asin', right_on='asin')

In [58]:
# Slicing the significant columns
DF4 = DF3.loc[:, ['overall','verified', 'reviewTime', 'asin', 'summary', 'unixReviewTime', 'title', 'brand', 'rank', 'price', 'description']]

In [59]:
# Converting the price column from string to float
DF4 = DF4.dropna(subset = ['price']) # Drop the rows containing NaN values for price 
DF4 = DF4[~DF4['price'].str.contains('-')] # Drop the rows containing "-" values for price (e.g: "123 - 158")
DF4['price'] = DF4['price'].str.replace(',', '') # Deleting "," for price (e.g: "$1,538.34")
DF4['price'] = DF4['price'].str.replace('$', '').astype(float) # Deleting "$" for price (e.g: " $1538.34")

In [60]:
# Loc the rows with price above $100
DF4 = DF4.loc[(DF4['price'] > 100), :]

In [61]:
DF4[['verified']] = DF4[['verified']].astype(str) 

In [62]:
# Drop all rows containing False in 'verified'
DF4 = DF4[~DF4['verified'].str.contains('False')] 

In [63]:
DF4.head()['reviewTime']

2298     11 8, 2011
2301     07 6, 2010
2302    09 21, 2016
2303    09 10, 2016
2304     09 4, 2016
Name: reviewTime, dtype: object

In [71]:
# add a time stamp 
DF4.loc[:, "timestamp"] = pd.to_datetime(DF4["reviewTime"],format="%m %d, %Y")

In [65]:
DF4.head()[["reviewTime", "timestamp"]]

,reviewTime,timestamp
2298,"11 8, 2011",2011-11-08
2301,"07 6, 2010",2010-07-06
2302,"09 21, 2016",2016-09-21
2303,"09 10, 2016",2016-09-10
2304,"09 4, 2016",2016-09-04


In [73]:
# create new columns for year, month and day
DF4.loc[:, "review_year"] = DF4["timestamp"].dt.year
DF4.loc[:, "review_month"] = DF4["timestamp"].dt.month
DF4.loc[:, "review_day"] = DF4["timestamp"].dt.day

In [75]:
# set timestamp as index
DF4.set_index("timestamp", inplace=True)

In [85]:
DF4.head().T

timestamp,2011-11-08,2010-07-06,2016-09-21,2016-09-10,2016-09-04
overall,4,5,5,5,5
verified,True,True,True,True,True
reviewTime,"11 8, 2011","07 6, 2010","09 21, 2016","09 10, 2016","09 4, 2016"
asin,9792372326,9792372326,9792372326,9792372326,9792372326
summary,K10 vs Mackie SRM450v2 vs Klipsch bookshelf,The best powered speaker so far!,Five Stars,Very impressive,Five Stars
unixReviewTime,1320710400,1278374400,1474416000,1473465600,1472947200
title,"QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10"""
brand,QSC,QSC,QSC,QSC,QSC
rank,">#66,697 in Musical Instruments (See Top 100 i...",">#66,697 in Musical Instruments (See Top 100 i...",">#66,697 in Musical Instruments (See Top 100 i...",">#66,697 in Musical Instruments (See Top 100 i...",">#66,697 in Musical Instruments (See Top 100 i..."
price,125.99,125.99,125.99,125.99,125.99
